In [16]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib notebook
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
import csv
import os
import glob


In [17]:
#rings ranges
#start is 2897 but depth stabilises at 2937. ends at 4796
#Seems to only be three rings in barrel survey rings dataset, barrel survey far also has some rings taken in positive depths
#thate are also strips of the detector top to bottom taken which could be more useful?

#barrel survey far starts at 8157, 3 rings from  15 to 7m depth
#i dont thing the whole thing is done? strips run from 0 to 180 degrees
rings = [[2937, 3466], [3517, 4086], [4147, 4656]]
depths = [-9.76, -1.44, 3.28]
#validation data:

val_coords = {'755':{'vert_lines':[[1703, 2], [3110, 3]], 'hor_lines':[1273]},
'756': {'vert_lines':[[1933, 3]], 'hor_lines':[1299]},
'757': {'vert_lines':[[910, 3], [2293, 4]], 'hor_lines':[1266]},
'758': {'vert_lines':[[1450, 4], [2750, 5]], 'hor_lines':[1300]},
'759': {'vert_lines':[[1820, 4], [3175, 5]], 'hor_lines':[1270]},
'760': {'vert_lines':[[2000, 5], [3245, 6]], 'hor_lines':[1883]},
'761': {'vert_lines':[ [2345, 5]], 'hor_lines':[1300]},
'762': {'vert_lines':[[1270, 5], [2576, 6]], 'hor_lines':[1266]},
'763': {'vert_lines':[[1706, 6], [3026, 7]], 'hor_lines':[2203]},
'764': {'vert_lines':[[690, 6], [2100, 7]], 'hor_lines':[1259]},       
'765': {'vert_lines':[[1050, 7], [2450, 8]], 'hor_lines':[1259]},
'766': {'vert_lines':[[1463, 8], [2886, 9]], 'hor_lines':[1256]},
'767': {'vert_lines':[[1800, 9], [3129,10]], 'hor_lines':[1250]},
'768': {'vert_lines':[[729, 9], [2100, 10], [3460, 11]], 'hor_lines':[1263]},     
'769': {'vert_lines':[[1070, 10], [2383, 11]], 'hor_lines':[1266]},
'770': {'vert_lines':[[1383, 11], [2733, 12]], 'hor_lines':[1276]},
'771': {'vert_lines':[[1575, 12], [2883, 13]], 'hor_lines':[1250]},
'772': {'vert_lines':[[2046, 12], [3350, 13]], 'hor_lines':[1263]},
'773': {'vert_lines':[[950, 12], [2306, 13]], 'hor_lines':[1246]},
'774': {'vert_lines':[[1463, 13], [2783, 14]], 'hor_lines':[1256]},
'775': {'vert_lines':[[1789, 14], [3033, 15]], 'hor_lines':[1236]},
'776': {'vert_lines':[[700, 14], [1956, 15], [3416, 16]], 'hor_lines':[1273]},
'777': {'vert_lines':[[1000, 15], [2400, 16]], 'hor_lines':[1253]},
'778': {'vert_lines':[[1370, 16], [2904, 17]], 'hor_lines':[1845]},
'779': {'vert_lines':[[1770, 17], [3316, 18]], 'hor_lines':[1220]},
'780': {'vert_lines':[[641, 18], [2123, 19]], 'hor_lines':[1250]},
'781': {'vert_lines':[[950, 19], [2441, 20]], 'hor_lines':[1250]},
'782': {'vert_lines':[[1370, 20], [2936, 21]], 'hor_lines':[1236]}, #these past few have been really good
'783': {'vert_lines':[[1787, 21], [3320, 22]], 'hor_lines':[2000]},
'784': {'vert_lines':[[600, 22], [2150, 23]], 'hor_lines':[1243]}, #differs by 20 deg so mayb 24, 25
'785': {'vert_lines':[[945, 23], [2483,24]], 'hor_lines':[1237]},              
'786': {'vert_lines':[[1293, 24], [2809, 25]], 'hor_lines':[1250]},
'787': {'vert_lines':[[1630,25], [3120, 26]], 'hor_lines':[1250]}, #maybe wronf
'788': {'vert_lines':[[730, 25], [2173, 26]], 'hor_lines':[1219]},
'789': {'vert_lines':[[750, 27], [2360, 28]], 'hor_lines':[1237]}, #18 degrees
'790': {'vert_lines':[[1203, 28], [2810, 29]], 'hor_lines':[1256]},
'791': {'vert_lines':[[2620, 29], [4283, 30]], 'hor_lines':[1250]},
'791': {'vert_lines':[[1679, 29], [3320, 30]], 'hor_lines':[1987]},  #Double check!!!!!!!!!!!!!!!!!!!!!
'792': {'vert_lines':[[560, 29], [2150, 30], [3000, 31]], 'hor_lines':[1266]}, #half super module
'793': {'vert_lines':[[816, 30], [1600, 31], [3154, 32]], 'hor_lines':[1266]}, #half super module
'794': {'vert_lines':[[600, 31], [2083, 32]], 'hor_lines':[1250]},
'795': {'vert_lines':[[908, 32], [2412, 33]], 'hor_lines':[1300]},
'796': {'vert_lines':[[1250, 33], [2816, 34]], 'hor_lines':[1250]},
'797': {'vert_lines':[[1725, 34], [3191, 35]], 'hor_lines':[1240]},
'798': {'vert_lines':[[570, 34], [2050, 35]], 'hor_lines':[1250]},
'799': {'vert_lines':[[1100, 34], [2450, 35]], 'hor_lines':[1250]},
'800': {'vert_lines':[[1429, 35], [2700, 36]], 'hor_lines':[1236]},
'901': {'vert_lines':[[1720, 36], [3158, 37]], 'hor_lines':[1236]},
'902': {'vert_lines':[[570,36], [2078, 37]], 'hor_lines':[1279]},    #64 lines above here
'903': {'vert_lines':[[937,37], [2370, 38]], 'hor_lines':[1279]},    
'904': {'vert_lines':[[1250,38], [2620, 39]], 'hor_lines':[1279]},    
'905': {'vert_lines':[[1687,38], [3000, 39 ]], 'hor_lines':[1279]},    

    }
val_names = val_coords.keys()


In [18]:
#image preprocessing functions
#if usig BGR masks, not needed if regular grayscale ones are used
def ConvertMasksBGR(BGR_img) :
    lower_pmts = np.array([250, 0, 0], dtype = "uint16")
    upper_pmts = np.array([255,0,0], dtype = "uint16")
    pmt_mask = cv2.inRange(BGR_img, lower_pmts, upper_pmts)
    pmt_mask = cv2.cvtColor(pmt_mask, cv2.COLOR_GRAY2BGR)
   # print(pmt_mask.shape)
    pmt_mask[pmt_mask == 255] = 1
    return pmt_mask



In [19]:
####################################################################################

def UndistortImage(orig) :
    #undistorted = orig.copy();

 # //Mat cameraMatrix = Mat::eye( 3, 3, CV_64F ); //K
 # //Mat distCoeffs = Mat::zeros( 1, 4, CV_64F ); //D
 # //  Here's the constants extracted directly from MATLAB:
 # //           FocalLength: [2.7605e+03 2.7670e+03]
 # //        PrincipalPoint: [1.9143e+03 1.5964e+03]
 # //             ImageSize: [3000 4000]
 # //      RadialDistortion: [-0.2398 0.1145]
 # //  TangentialDistortion: [0 0]
 # //               Skew: 0
 # //  
 # // The intrinsics fx, fy, cx and cy in your intrinsics matrix are the
 # // focal length and principle point above, and are dependant on the
  #// image size so best to make sure the images you're using haven't
  #// been rescaled.
  ##// The four values only give the minimum k1, k2, p1, p2 that you need 
  #// because calibrating additional parameters weren't enabled by the 
  #// students when they did this calibration. Tangential distortion is set to 
  #// zero because it was also disabled.  

    cameraMatrix = np.array([[ 2.7605e+03,      0,          1.9143e+03],
                        [0,               2.7670e+03,      1.5964e+03], 
                        [0,                    0,               1]] )
    
    distCoeffs = np.array( [-0.2398, 0.1145, 0, 0] )
 #   print(cameraMatrix)
    undistorted = cv2.undistort(orig, cameraMatrix, distCoeffs);

    return undistorted;


In [20]:
#stolen from stack overflow 
#this is set up for a multi peak gaussian fit 
def peaks_func(x, *params):
    #print(params)
    y = np.zeros_like(x)
    for i in range(0, len(params), 3):
        ctr = params[i]
        amp = params[i+1]
        wid = params[i+2]
        y = y + amp * np.exp( -((x - ctr)/wid)**2)
    return y


In [21]:

#guess locations of peaks using a threshold and data 
#instead of fitting to peaks, just see where peak goes above and below average and get centre of those two points

def peak_guess(x_dat, y_dat) : 
    tempx = []
    tempy = []
    peak_loc = []
    
    #if it is above this it is a peak
    #peak_thresh = np.min(y_dat) + (np.max(y_dat) - np.min(y_dat))*0.5
    peak_thresh = np.average(y_dat)
    for i in range(len(y_dat)) :
        if y_dat[i] >= peak_thresh :
            on_peak = True
            tempx.append(x_dat[i])
            tempy.append(y_dat[i])
        else :
        #should only do this AFTER end of peak
            if len(tempx) > 10 : #no empty data
                #print([tempy[0], tempy[-1], len(tempy)])

                #if int(np.abs(tempy[0] - tempy[-1])) < 500: #check that it is a full peak
                peak_start = tempx[0]
                peak_end = tempx[-1]
                peak_cen = peak_start + (peak_end - peak_start) / 2
                peak_max = np.max(tempy)
                peak_wid = peak_end - peak_start

                peak_loc.append([peak_cen, peak_max, peak_wid])
            tempx = []
            tempy = []
    return peak_loc, peak_thresh

####################################################################################


In [22]:
#drawing various lines 
#peaks is the pmt lines, margins are the image crops that remove the edges of images and need to be accounted for 
#super module lines are recalculated in here

def draw_lines(img_in, peaks1, peaks2, margin1, margin2, super_x, super_y, draw_pmt_lines = True) :
    global smod_count_num
    global smod_count_lines
    
    #make a new instance, may be unnecessary but really dont want to draw over data 
    img = img_in.copy()
    if draw_pmt_lines == True :
        if len(peaks1) > 9 :
            num_rem1 = 2
        else :
            num_rem1 = 1
        
        if len(peaks2) > 9:
            num_rem2 = 2
        else :
            num_rem2 = 1
                       
            
       # cv2.line(img, (0, int(peaks1[0])), (img.shape[1], int(peaks1[0])),  color=[255, 0, 255] , thickness=2)
       # cv2.line(img, (0, int(peaks1[-1])), (img.shape[1], int(peaks1[-1])),  color=[255, 0, 255] , thickness=2)

        for i in range(0, len(peaks1)) :
            if (i < num_rem1) or (i >= len(peaks1)-num_rem1):
                cv2.line(img, (0, int(peaks1[i]) + margin1), (img.shape[1], int(peaks1[i] + margin1)),  color=[255, 0, 255] , thickness=2)
            else :
                cv2.line(img, (0, int(peaks1[i]) + margin1), (img.shape[1], int(peaks1[i] + margin1)),  color=[0, 0, 255] , thickness=2)

        for j in range(0, len(peaks2)) :
            if (j < num_rem2) or (j >= len(peaks2)-num_rem2) :
                cv2.line(img, (int(peaks2[j] + margin2), 0), (int(peaks2[j] + margin2), img.shape[0]),  color=[255, 0, 255] , thickness=2) #PUT IN ANGLE CORRECTION
            else :
                cv2.line(img, (int(peaks2[j] + margin2), 0), (int(peaks2[j] + margin2), img.shape[0]),  color=[0, 0, 255] , thickness=2)
                 
    
    for k in range(0, len(super_x)) :
        cv2.line(img, (0, int(super_x[k] + margin1)), (img.shape[1], int(super_x[k] + margin1)),  color=[0, 255, 0] , thickness=4)
        
    for l in range(0, len(super_y)) :
        
        cv2.line(img, (int(super_y[l] + margin2), 0), (int(super_y[l] + margin2), img.shape[0]),  color=[0, 255, 0] , thickness=4)
        #cv2.putText(img, f'{smod_count_num}', (int(super_y[l] + margin2), int(img.shape[0]/2 + margin2)), cv2.FONT_HERSHEY_SIMPLEX, 3, (0,255,0), 2)
        #smod_count_lines.append([super_y[l], smod_count_num])
        #smod_count_num += 1
            
    return img



In [23]:
######Questionable methods begin here
def super_lines(pmt_lines_long, axis, prev_lines, do_override=False) :
    
    #_long is all lines, remove outer lines as they can be wrong
    super_mod_lines = []
    if len(pmt_lines_long) > 9 :
        num_rem = 2
    else :
        num_rem = 1

    pmt_lines = pmt_lines_long[num_rem:-num_rem]
    pmt_width_punt = np.average(np.diff(pmt_lines))
    centre_prev = []

    #IT WOULD BE NICE TO GET AN IMAGE INDEPENDEDNT MEASURE OF GAPS E.G. BASED OFF OF PMT WIDTH  
    gaps_long = np.diff(pmt_lines_long)
    gaps = np.diff(pmt_lines)
    
    #Pattern test horizontal lines
    #have three possibilities: 1st, 2nd, 3rd gap is a super module
    #split gap dat into predicted large and small gaps
    #small gap data is more reliable as there are more of them
    #True small gap data will have small difference between gap sizes, false ones will contain large and small gaps so difference will be larger
    #there is probably a case where this is not true
    if axis == "x" :
        #hypothesis that first gap is super module
        
        hyp1 = {'sindx':range((num_rem)%3,len(gaps_long), 3), 'hi':gaps[::3], 'lo':np.setdiff1d(gaps,gaps[::3]), 
                'long_hi':gaps_long[(num_rem)%3::3], 'prev_diffs':[]}
        hyp2 = {'sindx':range((1 + num_rem)%3,len(gaps_long), 3), 'hi':gaps[1::3], 'lo':np.setdiff1d(gaps,gaps[1::3]), 
                'long_hi':gaps_long[(num_rem+1)%3::3], 'prev_diffs':[]}
        hyp3 = {'sindx':range((2 + num_rem)%3,len(gaps_long), 3), 'hi':gaps[2::3], 'lo':np.setdiff1d(gaps,gaps[2::3]),
                'long_hi':gaps_long[(num_rem+2)%3::3], 'prev_diffs':[]}
        
        hyps = [hyp1, hyp2, hyp3]
 #       print(hyps)
        scores =[] 
        for i in range(len(hyps)):            

            max_width = np.average(hyps[i]['hi'])
            average_lo = np.average(hyps[i]['lo'])
            
            scores.append(max_width) #seems all that is necessary for horizontal
            
        #TODO: update this when have projection horizontally
        if prev_lines == None :
            best = scores.index(max(scores))
        else :
            best = scores.index(max(scores))

                    
            

    #Simpler: average of large gap data will be larger than average of small gap
    #more gaps avai;lable so I think this is safe
    if axis == "y" :
        # for every other gap (modules)
      #  hyp1 = {'sindx':range(0,len(gaps), 2),'hi':gaps[::2], 'lo':np.setdiff1d(gaps,gaps[::2])}
      #  hyp2 = {'sindx':range(1,len(gaps), 2),'hi':gaps[1::2], 'lo':np.setdiff1d(gaps,gaps[1::2])}
      #  hyps = [hyp1, hyp2]
      #  if np.average(hyp1['hi']) > np.average(hyp2['hi']) :
      #      best = 0
      #  else :
      #      best = 1
    
    
    
        #mark supermodules like previously
        #sindx: index of proposed super module gaps in long gaps array
        #hi: proposed widths of large gaps with edges removed
        #lo: proposed gaps of small gaps with edges removed
        #long_hi: widths in long array
        #prev_diffs: difference between large gaps and ones from previous image
        
        hyp1 = {'sindx':range((num_rem)%4,len(gaps_long), 4),'hi':gaps[::4], 'lo':np.setdiff1d(gaps,gaps[::4]), 
                'long_hi':gaps_long[(num_rem)%4::4], 'prev_diffs':np.full(len(range((num_rem)%4,len(gaps_long), 4)), 10000., dtype=float)}
        hyp2 = {'sindx':range((1 + num_rem)%4,len(gaps_long), 4),'hi':gaps[1::4], 'lo':np.setdiff1d(gaps,gaps[1::4]), 
                'long_hi':gaps_long[(num_rem+1)%4::4], 'prev_diffs':np.full(len(range((1+num_rem)%4,len(gaps_long), 4)), 10000., dtype=float)}
        hyp3 = {'sindx':range((2 + num_rem)%4,len(gaps_long), 4),'hi':gaps[2::4], 'lo':np.setdiff1d(gaps,gaps[2::4]),
                'long_hi':gaps_long[(num_rem+2)%4::4], 'prev_diffs':np.full(len(range((2+num_rem)%4,len(gaps_long), 4)), 10000., dtype=float)}
        hyp4 = {'sindx':range((3 + num_rem)%4,len(gaps_long), 4),'hi':gaps[3::4], 'lo':np.setdiff1d(gaps,gaps[3::4]),
                'long_hi':gaps_long[(num_rem+3)%4::4], 'prev_diffs':np.full(len(range((3+num_rem)%4,len(gaps_long),4)), 10000., dtype=float)}
        
        hyps = [hyp1, hyp2, hyp3, hyp4]

        #print(gaps)
        #print(hyp1['lo'])
        #print(hyp2['lo'])
        #print(hyp3['lo'])
        #print(hyp4['lo'])

        
        max_width_scores =[] 
        average_lo_scores =[] 
        min_to_prev_scores =[] 
        values = []
        scores = np.zeros(4)
        for j in range(len(hyps)):
            #scores.append(np.max(hyps[j]['lo']) - np.min(hyps[j]['lo']))
            max_width_scores.append(np.max(hyps[j]['hi'])) 
          #  average_lo_scores.append(np.average(hyps[j]['lo']))
          #  hyp_score = [j, np.max(hyps[j]['hi']), -np.average(hyps[j]['lo'])] #negative average stored so it can be maximised
            

            #score for disrtance from past lines
            print('scores for compare', prev_lines, hyps[j]['prev_diffs'], hyps[j]['sindx'])
            if prev_lines != None:
                for l in range(len(prev_lines )):
                    for k in range(len(hyps[j]['sindx'])):
                        #print('parts', l, k)
                        nearest_prev = abs((pmt_lines_long[hyps[j]['sindx'][k]]+200) - prev_lines[l])
                        if nearest_prev < hyps[j]['prev_diffs'][k] :
                            hyps[j]['prev_diffs'][k] = nearest_prev
                       # print(k - l)
                    
                    
                min_to_prev_scores.append(np.min(hyps[j]['prev_diffs']))
               # hyp_score.append(np.min(hyps[j]['prev_diffs']))
                

        #best hyp should have lowest score as it is in 1st place for most metrics
        print(f'hyp Scores:{scores}')
        #print(f'hyp Vals comp:{vals_compare}')
        #best = np.argmin(scores)
        
        #best = np.argmin(average_lo_scores)
        best = np.argmax(max_width_scores)
        
        #check if its fallen halfway between boundaries
        if prev_lines != None:
            if np.min(hyps[best]['prev_diffs']) > pmt_width_punt*0.75 :
                print(f'Found lines a long way from previous ones')
                #find the next module boundary
                next_best =  (best + 2)%4

                print([np.max(hyps[next_best]['hi']), 0.9*np.max(hyps[best]['hi']), np.min(hyps[next_best]['prev_diffs']), np.min(hyps[best]['prev_diffs'])])

                if (np.max(hyps[next_best]['hi']) > 0.9*np.max(hyps[best]['hi'])) and ( np.min(hyps[next_best]['prev_diffs'])< 1.0*pmt_width_punt) :
                    print(f'Changed best from {best} to {next_best}')
                    best = next_best
                else:
                    print(f'Line is still in disagreement')

        #override:
        if supervised == True :
            if do_override == True :
                #make window for compare
                cv2.namedWindow('override',cv2.WINDOW_NORMAL) 
                cv2.moveWindow('override', 500, 0)
                cv2.resizeWindow('override', 1200, 900)
                #read image
                print(img_path)
                temp_for_override = cv2.imread(img_path)
                for k in hyps[best]['sindx'] :
                    if k != len(pmt_lines_long) :
                        temp_line = pmt_lines_long[k] + (pmt_lines_long[k+1] - pmt_lines_long[k])/2
                        cv2.line(temp_for_override, (int(temp_line)+200, 0), (int(temp_line)+200, temp_for_override.shape[0]), color=[255, 255, 255], thickness=3)

                cv2.imshow('override', temp_for_override)

                cv2.waitKey(0)
                print_vals = hyps[best]['sindx']
                print(f'best is {best} : {print_vals} Override?')
                override_decision= input("(y/n)\n-->")
                if(override_decision.lower() == "y"):
                    best_override = input('Set new lines (between 0 and 3):')

                    if  int(best_override.lower()) < 4 :
                        best = int(best_override.lower())
                    else :
                        print('Invalid override')

       # print(best)

    
    for k in hyps[best]['sindx'] :
        if k != len(pmt_lines_long) :
            super_mod_lines.append(pmt_lines_long[k] + (pmt_lines_long[k+1] - pmt_lines_long[k])/2)
    
    return super_mod_lines, hyps[best]['hi']
#################################################################
        

In [24]:
def get_scores(img, angle) :
    #get grayscale so only 2d matrix
    gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    img_size = gray_img.shape
    x_score = []
    y_score = []
    mult = 2    #how many pixel lines skipped in image scan (1 is dto get a score for every line)


    #loop over a and y with lines drawn on a mask to get score from pixel sums
   # print(img_size)
    for i in np.linspace(0, img_size[0], int(img_size[0]/mult)) :
        img_mask = np.zeros(img_size,dtype=np.uint8)
        start_y = int(i)
        end_y = start_y + int(angle*img_size[1])  #minus because coords measured from top left corner
        cv2.line(img_mask, (0, start_y), (img_size[1], end_y),  color=1 , thickness=1)


        fit = gray_img*img_mask
        
        #DEBUG
        #print(np.unique(fit))
        #if(fit.all()<1) :
        #    continue     
        #init_rad = circ_r
      # cv2.imshow("img", gray_img)
      # cv2.waitKey(0)
    
        score = np.sum(fit)
        x_score.append(score)
        

    for j in np.linspace(0, img_size[1], int(img_size[1]/mult)) :
        img_mask = np.zeros(img_size,dtype=np.uint8)
        start_x = int(j)
        end_x = start_x - int(angle*img_size[0])
        cv2.line(img_mask, (start_x, 0), (end_x, img_size[0]),  color=1 , thickness=1)

        fit = gray_img*img_mask

        score = np.sum(fit)
        y_score.append(score)

    return x_score, y_score

In [25]:
def plot_data(x1, y1, pmt_lines_1, x2, y2, pmt_lines_2) :

    #IT WOULD BE NICE TO GET AN IMAGE INDEPENDEDNT MEASURE OF GAPS E.G. BASED OFF OF PMT WIDTH  
    gaps1 = np.diff(pmt_lines_1)
    gaps2 = np.diff(pmt_lines_2)
    gap_x_1 = []
    gap_x_2 = []
    
    for k in range(len(gaps1)) :
        gap_x_1.append(pmt_lines_1[k] + gaps1[k]/2)
        
    for l in range(len(gaps2)) :
        gap_x_2.append(pmt_lines_2[l] + gaps2[l]/2)
    
    #plot setup
    fig, axs = plt.subplots(2, 2, figsize=(10,6), sharex='col')
    fig.tight_layout()
    plt.subplots_adjust(left = 0.1, bottom = 0.1, hspace=0.4, wspace=0.2)


    axs[0, 0].plot(x1, y1) #line scores
    for i in pmt_lines_1 :
        axs[0, 0].axvline(x=i)
    #axs[0, 0].axhline( thresh1, color = 'g', label='thresh')
    axs[0,0].set_title("Horizontal line scan")
    axs[0,0].set_xlabel("X position (px)")
    axs[0,0].set_ylabel("Score (px)")

    
    axs[0, 1].plot(x2, y2) 
    for j in pmt_lines_2 :
        axs[0, 1].axvline(x=j)

    #axs[0, 1].axhline( thresh2, color = 'g', label='thresh')
    axs[0, 1].set_title("Vertical line scan")
    axs[0,1].set_xlabel("Y position (px)")
    axs[0,1].set_ylabel("Score (px)")

    #decision polots
    #TODO: put predicted lines on here?
    axs[1, 0].set_title("Horizontal super module decision")
    axs[1, 0].scatter(gap_x_1, gaps1, label='gaps')
    axs[1, 0].axhline( np.median(gaps1), color = 'r', label='median')
    axs[1, 0].axhline( np.average(gaps1), color = 'b', label='average')
    axs[1, 0].legend(loc="upper right", prop={'size': 6})
    axs[1,0].set_xlabel("X position (px)")
    axs[1,0].set_ylabel("Gap size (px)")

    axs[1, 1].set_title("Vertical super module decision")
    axs[1, 1].scatter(gap_x_2, gaps2, label='gaps' )
    axs[1, 1].axhline( np.median(gaps2), color = 'r', label='median')
    axs[1, 1].axhline( np.average(gaps2), color = 'b', label='average')
    axs[1, 1].legend(loc="upper right", prop={'size': 6})
    axs[1,1].set_xlabel("Y position (px)")
    axs[1,1].set_ylabel("Gap size (px)")

    axs[0, 0].tick_params(reset=True)
    axs[0, 1].tick_params(reset=True)


In [26]:
def get_hor_smod_size(super_y_lines, peaks_y_lines) :
    #get size of super module in the image, difficult case when only 1 smod found, have to do 4*pmt width 
    #which is less accurate
    #TODO: dont need to calculate both, just temporary to look at similarities
    if len(super_y_lines)>1:
        smod_size_1 = np.average(np.diff(super_y_lines))
        smod_size_2 = 4*np.average(np.diff(peaks_y_lines))
        smod_size = smod_size_1
    else:
        smod_size_1 = None
        smod_size_2 = 4*np.average(np.diff(peaks_y_lines))
        smod_size = smod_size_2

    print(f'smod sizes smod/pmt: {smod_size_1}, {smod_size_2}')
    return smod_size



In [27]:
#sometimes pixel scores hover around the threshhold, this tries to sort that out
#Very important bit of code, if a double PMT line gets through it messes up the labels
#takes list output from peaks_guess and returns same format

def fix_double_peaks(peaks) :
    #If there is a double peak in PMT lines, average the two
    peak = 0
    peaks_end = len(peaks) - 1
    #print(peak, len(peaks_x))
    while peak < peaks_end :
        if abs(peaks[peak] - peaks[peak + 1]) < 200 :
            print(f'Double Peak found: {peaks}, {peaks[peak]}, {peaks[peak + 1]}')
            peaks[peak] = (peaks[peak] + peaks[peak + 1])/2.0
            peaks.pop(peak + 1)
            print(f'Corrected: {peaks}')
            peaks_end = len(peaks) - 1
        else :
            peak += 1
    return peaks


In [28]:
def find_super_modules(indir, outdir, name,  shift_info=[[], 1]) :
    global smod_track_num
    global prev_img_end_num
    global failed_imgs
    global prev_hor_end_num
    global img_path
    
    #raw image
    img_path = f'{raw_img_dir}{name[:3]}.jpg'
    
    #segmented image
    img_in = cv2.imread(f'{indir}{name}.png')


    
    base_name = name[0:3]
    #if gray mask :
    #img_in[img_in == 2] = 0

    #if bgr mask :
    img_in = ConvertMasksBGR(img_in)


    #Undistort image
    if use_undistort == True :
        imge = UndistortImage(img_in)
    else :
        imge = img_in
    
    #choose output image to be recoloured mask, or overlaid on original
   # line_img = cv2.imread(f'/home/dm3315/Documents/SK/PMT_learning/images/Raw/neutsrv2.triumf.ca \
    #                      /neutshare/superk/TOW-Feb2020/Processed/BarrelSurveyRings_TB3 \
    #                      /BarrelSurveyRings_median/ring_avg_all/{name}.jpg')
    line_img = imge.copy()
    line_img[line_img < 1] = 100  #is using same input

    
    #Crop to central region
    hor_margin = 200 #cuts on x axis
    vert_margin = 100 #cuts on y axis
    imge = imge[hor_margin:-hor_margin, vert_margin:-vert_margin, :]
    
    #DEBUG
   # print(np.unique(imge))
    #cv2.imshow("img", imge)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    
    #gET PREVIOUS LINE INFO
    pred_vert_lines = shift_info[0]
    img_vert_lines = []     #[[super_y, num]...]
    img_hor_lines = []
    
    
    #Scan in x and y a line with the angle above and count the number of pixels that appear in the PMT mask
    x_line_score, y_line_score = get_scores(imge, 0)# angle)
    mult = 2 #for speed
    x1 = np.linspace(0, imge.shape[0], int(imge.shape[0]/mult))
    x2 = np.linspace(0, imge.shape[1], int(imge.shape[1]/mult))

    #Get approximate locations of peaks -  centre of pmt lines
    peak_loc_1, guess_thresh_1 = peak_guess(x1, x_line_score)
    peak_loc_2, guess_thresh_2 = peak_guess(x2, y_line_score)
    
    
    ##DEBUG
    #plt.plot(x1, x_line_score)
    #plt.plot(x2, y_line_score)
    #print(peak_loc_1)
    #print(peak_loc_2)
    
    
    print(f"Guess: {len(peak_loc_1)} horizontal gaps above {guess_thresh_1}, {len(peak_loc_2)} vertical above {guess_thresh_2}")

    
#Peak finding methods, three are implemented at the moment
#some more complicated  peak finding methods that took longer and more unstable so abandoned, kept in case necessary
###########################################################################################################
    #Do a gaussian fit with an inital guess from above
   # popt1, pcov1 = curve_fit(peaks_func, x1, x_line_score, p0=peak_loc_1)
   # popt2, pcov2 = curve_fit(peaks_func, x2, y_line_score, p0=peak_loc_2)

    #scipy function
    #peak_index_x, _ = find_peaks(np.array(x_line_score),  distance=100, prominence=500, width=10) #\
                                              #height=guess_thresh_1, \
                                              #\ 
                                 #np.abs(np.average(np.diff(peak_loc_1)))*0.5, 
                                               #\
                                              
    
    #peak_index_y, _ = find_peaks(np.array(y_line_score), distance=100, prominence=500, width = 10)
                                              #height=guess_thresh_2, \
                                               #\  #np.abs(np.average(np.diff(peak_loc_2)))*0.5, 
                                               #\
                                              

  #  print(peak_index_x)
  #  print(peak_index_y)
    
    #find peak centers with different methods
   # peaks_x_1 = x1[peak_index_x]
    #peaks_y_1 = x2[peak_index_y]
##########################################################################################

    #guess  method

    peaks_x_2 = []
    peaks_y_2 = []
    for i in peak_loc_1 :
        peaks_x_2.append(i[0])

    for i in peak_loc_2 :
        peaks_y_2.append(i[0])
    
    
    #FIT METHOD
    '''
    fit1 = peaks_func(x1, *popt1)
    fit2 = peaks_func(x2, *popt2)    
    peaks_x_3 = []
    peaks_y_3 = []
    for i in range(0, len(popt1), 3) :
        peaks_x_3.append(popt1[i])
    
    for j in range(0, len(popt2), 3) :
        peaks_y_3.append(popt2[j])
    '''    
    
    #Choose a method and double peaks
    peaks_x = fix_double_peaks(peaks_x_2)
    peaks_y = fix_double_peaks(peaks_y_2)
    
    
    
    #should only be the case for first image, also catches if drone moves too far in which case will likely be wrong
    if len(pred_vert_lines) == 0 :
        #Find super modules
        super_x, gaps_x = super_lines(peaks_x, "x", None, do_override=False)
        super_y, gaps_y = super_lines(peaks_y, "y", None, do_override=False)
        
        smod_size = get_hor_smod_size(super_y, peaks_y)
        
        for new_line in super_y :
            prev_img_end_num += 1
            img_vert_lines.append([new_line+ vert_margin, prev_img_end_num])
        for new_hor_line in range(len(super_x)) :
            temp_hor_num = prev_hor_end_num + new_hor_line
            img_hor_lines.append([super_x[new_hor_line]+hor_margin, temp_hor_num+1])
        
        #Draw lines on image for reference
        line_img_drawn = draw_lines(line_img, peaks_x, peaks_y, hor_margin, vert_margin, super_x, super_y,  draw_pmt_lines = True)           

    else :
        #initial guess with no previous lines (to get size
        super_x, gaps_x = super_lines(peaks_x, "x", None, do_override=False)
        super_y, gaps_y = super_lines(peaks_y, "y", None, do_override=False)
        
        #print(pred_vert_lines.shape)
        num_prev = len(pred_vert_lines)
        num_current = len(super_y)
        cor_lines = []   #predicted lines corrected for this image
        cor_num = []     #predicted line number
        

        
        smod_size = get_hor_smod_size(super_y, peaks_y)
        

        #get predicted previous lines
        for ucor_line in range(num_prev) :
                cor_lines.append(pred_vert_lines[ucor_line][0] * smod_size)
                cor_num.append(pred_vert_lines[ucor_line][1])
        
        #print(smod_size)
        #print(gaps_y)
        print(f'diff gaps_y:{np.diff(gaps_y)}')
        print(f'super_y:{super_y}')
        print(f'max_lines:{np.max(cor_lines)}')
        print(f'max_num:{np.max(cor_num)}')
        print(f'info:{pred_vert_lines}')
        print(f'old lines:{cor_lines}')
        print(f'new_lines {super_y}  {super_x}')


        max_prev_line = np.max(cor_lines)
        max_prev_num = np.max(cor_num)

        
        #Update super modules with previous lines
        #get super module lines with corrected lines as a punt
        super_x, gaps_x = super_lines(peaks_x, "x", cor_lines, do_override=True)
        super_y, gaps_y = super_lines(peaks_y, "y", cor_lines, do_override=True)    
    
    
        #Label smod lines
        #loop through lines in current image, see if a previous line matches, if so give it that number
        #if not increase the global most recent number and assign that number
        #if a line ends up having no number, call image a failure
        #not sure what to do in this case, maybe skip image?
        
        
        #horizontal lines
        hor_line_num = 1
        for hor_line in super_x :
            #TODO: generalise this and include tracking with depth, currently assume depth is constant
            img_hor_lines.append([hor_line+hor_margin, hor_line_num])
            hor_line_num += 1
        
        
        #vertical_lines
        #make an array of line positions to label them
        line_labels = np.zeros((2, len(super_y)), dtype=float)
        for line in range(len(super_y)) :
            line_labels[0, line] = super_y[line]
        
        print(f'Line_labels : {line_labels}')
        
        #Tracks if label numbers have been assigned to this image
        #If at the end of labelling no numbers are assigned, the count continues from the last image
       # nums_assigned = False
        lines_numbered = 0
        for line in range(len(super_y)) : #pred_vert_lines = [[line, number]..]
            #TODO: Track numbers from previous image
            num_assigned = False
            margin = np.average(np.diff(super_y))*(3/8) # is 1.5 pmts hopefully
            
            #line matches reprojected one -  reliable
            for prev_line in range(len(cor_lines)) :
                if abs(cor_lines[prev_line] - (super_y[line]+vert_margin)) < margin :
                    line_num = cor_num[prev_line]
                    line_labels[1, line] = line_num
                   # img_vert_lines.append([super_y[line]+vert_margin, cor_num[prev_line]])
                   # num_assigned = True
                    lines_numbered += 1
                    most_recent_line = [super_y[line]+vert_margin, cor_num[prev_line]]
                else :
                    print('no line coincidence')
        
        #if that hasnt worked, take the nearest lines to the left which will help cover reprojection errors
        if lines_numbered == 0 :
            for line in range(len(super_y)) : #pred_vert_lines = [[line, number]..]
                num_assigned = False
                for prev_line in range(len(cor_lines)):
                    #TODO update this to have a loose cut?
                    line_diff = cor_lines[prev_line] - (super_y[line] + vert_margin) 
                    if line_diff < 0 and abs(line_diff) < 800 and cor_lines[prev_line] > 0:
                        line_labels[1, line] = cor_num[prev_line]
                        num_assigned = True
                            
                if num_assigned :
                    lines_numbered += 1
        print(f'lines : {lines_numbered} {line_labels}')
        print('****')
        print(f'Have {line_labels}, {np.nonzero(line_labels[1,:])}, {lines_numbered}')
        #if at least one line matched reprojected lines, but not all of them have been, number the rest off of that
        if 0 < lines_numbered < len(super_y):
            while_kill = 0
            while len(np.nonzero(line_labels[1,:])[0]) != len(super_y) :
                while_kill += 1
                if while_kill > 100 :
                    print('No Resolution')
                    break
                    
                print(f'Have {line_labels}, {np.nonzero(line_labels[1,:])}')
                for line in np.nonzero(line_labels[1,:])[0] :
                    if line == 0 :
                        line_labels[1, 1] = line_labels[1, 0] + 1
                        print(f'Set line_labels[1, 1]: {line_labels[1, 1]} to {line_labels[1, 0] + 1}')
                    elif line == len(super_y)-1 :
                        line_labels[1, -2] = line_labels[1, -1] - 1
                        print(f'Set line_labels[1, -2]: {line_labels[1, -2]} to {line_labels[1, -1] -1}')

                    else :
                        line_labels[1, line - 1] = line_labels[1, line] - 1
                        print(f'Set line_labels[1, {line}-1]: {line_labels[1, line - 1]} to {line_labels[1, line] -1}')

                        line_labels[1, line + 1] = line_labels[1, line] + 1
                        print(f'Set line_labels[1, {line}+1]: {line_labels[1, line + 1]} to {line_labels[1, line] +1}')
        
        print('****')
        print(f'lines: {lines_numbered}')
        #if still notheing, image has failed, give numbers continuing from previous image
        if lines_numbered == 0 :
            failed_imgs.append(f'{base_name}')
            print('IMAGE FAILED')
            for line in range(len(super_y)):
                line_labels[1, line] = max_prev_num + line
                
                
        #Append the lines and numbers to the data
        for line in range(len(super_y)):
            img_vert_lines.append([line_labels[0, line], line_labels[1, line]]) 
            
    #TODO: if image failes, skip image? manually ammend?
                
     #Draw lines on image for reference
        line_img_drawn = draw_lines(line_img, peaks_x, peaks_y, hor_margin, vert_margin, super_x, super_y,  draw_pmt_lines = True)               
                
        
        #draw_previous numbers and lines passed through
        for num_prev_line in range(len(cor_lines)) :
            cv2.line(line_img_drawn, (int(cor_lines[num_prev_line]), 0), (int(cor_lines[num_prev_line]), line_img.shape[0]),  color=[255, 255, 255] , thickness=2)
            cv2.putText(line_img_drawn, f'{cor_num[num_prev_line]}', (int(cor_lines[num_prev_line]), int(line_img.shape[0]/3)), \
                        cv2.FONT_HERSHEY_SIMPLEX, 3, (255,255,255), 2)
    
    



    
    #draw current numbers from returned values (img_vert_lines)
    for num_current_line in img_vert_lines:
        cv2.putText(line_img_drawn, f'{num_current_line[1]}', (int(num_current_line[0]), int(200)), \
                    cv2.FONT_HERSHEY_SIMPLEX, 3, (0,255,0), 2)

    for num_current_line in img_hor_lines:
        cv2.putText(line_img_drawn, f'{num_current_line[1]}', (200, int(num_current_line[0])), \
                    cv2.FONT_HERSHEY_SIMPLEX, 3, (0,255,0), 2)
        
    #Label PMTs
    #TODO: move array converstion up
    img_hor_lines = np.asarray(img_hor_lines)   #smod horizontal lines
    img_vert_lines = np.asarray(img_vert_lines)   #smod vertical lines
    
    peaks_x_cor = np.array([ucor_x + hor_margin for ucor_x in peaks_x])   #pmt horizontal lines
    peaks_y_cor = np.array([ucor_y + vert_margin for ucor_y in peaks_y])  #pmt vertical lines
   # print(f'hor_lines : {img_vert_lines[:,0]}')
    
    #write locations and labels to file to pass to ellipse fitter:
    pmt_label_file = open(f'{outdir}pmt_loc_label.txt', 'a+')
    
   # Work out the labels for pmts in the images based 
    id_info = np.zeros((len(peaks_x), len(peaks_y), 6)) # for each crossing of pmt lines: [xpos, ypos, smod_x, smod_x, smod_y, smod_pos_x, smod_pos_y]
    for x in range(len(peaks_x)) :
        for y in range(len(peaks_y)) :
            id_info[x, y, 0] = peaks_x_cor[x]
            id_info[x, y, 1] = peaks_y_cor[y]
            
            #Super module ID
            #horizontal number
            if peaks_x_cor[x] < np.min(img_hor_lines[:,0]) :
                id_info[x, y, 3] = img_hor_lines[0,1] - 1
                id_info[x, y, 5] = 3 - np.size(np.where(peaks_x_cor < np.min(img_hor_lines[:,0])))  + x

            else :
                indx = np.max(np.where(img_hor_lines[:,0] < peaks_x_cor[x]))
               # print(f'indx x {indx} {img_hor_lines[indx, 1]} {np.where(img_hor_lines[:,0] < peaks_x_cor[x])}')
                id_info[x, y, 3] = img_hor_lines[indx, 1]
                id_info[x, y, 5] = np.size(np.where((peaks_x_cor > img_hor_lines[indx,0]) & (peaks_x_cor < peaks_x_cor[x])))

            #vertical line numbering
            if peaks_y_cor[y] < np.min(img_vert_lines[:,0]) :
                id_info[x, y, 2] = img_vert_lines[0,1] - 1
                id_info[x, y, 4] = 4 - np.size(np.where(peaks_y_cor < np.min(img_vert_lines[:,0])))  + y

            else : 
                indx = np.max(np.where(img_vert_lines[:,0] < peaks_y_cor[y]))
                #print(f'indx y {indx} : {img_vert_lines[indx, 0]} : {np.where((img_vert_lines[indx,0]< peaks_y_cor))} : {np.where((img_vert_lines[indx,0]< peaks_y_cor) & (peaks_y_cor < peaks_y_cor[y]))}')
                #print(f'where y {np.where((img_vert_lines[indx,0]< peaks_y_cor) & (peaks_y_cor < peaks_y_cor[y]))}')
                id_info[x, y, 2] = img_vert_lines[indx, 1]
                id_info[x, y, 4] = np.size(np.where((img_vert_lines[indx,0]< peaks_y_cor) & (peaks_y_cor < peaks_y_cor[y])))
            
            #Write to file
            pmt_label_file.write(f'{base_name}:{id_info[x, y, 0]}:{id_info[x, y, 1]}:{id_info[x, y, 2]}:{id_info[x, y, 3]}:{id_info[x, y, 4]}:{id_info[x, y, 5]} \n')

    pmt_label_file.close()
    
    
    #draw pmt ids
    for b in id_info :
        for a in b :
            #print(a)
            cv2.putText(line_img_drawn, f'{a[2:]}', (int(a[1])-100, int(a[0])+100), \
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

    
    #Write peak locations and super modules to file for an easy guess
    #[name:pmt_locs_x:pmt_locs_y:super_x:super_y]
    pmt_loc = f'{base_name}:{[x+hor_margin for x in peaks_x]}:{[y+vert_margin for y in peaks_y]}:{super_x}:{super_y}'
    pmt_file = open(f'{outdir}pmt_loc.txt', 'a+')
    pmt_file.write(f'{pmt_loc} \n')
    pmt_file.close()
    

    
    
    
    #compare to validation and draw lines
    num_cor = 0
    if base_name in val_coords :
        for i in super_x :
            for j in val_coords[base_name]['hor_lines'] :
                #print(f'vert:{i} {j}')
                cv2.line(line_img_drawn, (0, j), (line_img.shape[1], j),  color=[255, 0, 0] , thickness=2)

        
                if abs(i - (j-hor_margin)) < 200 :
                    print([i, (j-hor_margin)])
                    num_cor += 1
        
        for k in super_y :
            for l in val_coords[base_name]['vert_lines'] :
                cv2.line(line_img_drawn, (l[0], 0), (l[0], line_img.shape[0]),  color=[255, 0, 0] , thickness=2) #PUT IN ANGLE CORRECTION
                cv2.putText(line_img_drawn, f'{l[1]}', (int(l[0]), int(500)), \
                    cv2.FONT_HERSHEY_SIMPLEX, 3, (255,0,0), 2)

                #print(f'hor:{k} {l}')
                if abs(k - (l[0]-vert_margin)) < 200 :
                    print([k, (l[0]-vert_margin)])
                    num_cor += 1                    
        #print(num_cor)
    #plot all PMT lines
    print(f"Found {len(gaps_x)} horizontal gaps")
    print(f"Found {len(gaps_y)} vertical gaps")

    num_modules =  len(gaps_y)
   # print(super_x)

   # plt.show()
    cv2.imwrite(f"{outdir}{name}_lines.png", line_img_drawn)
    
    plot_data(x1, np.array(x_line_score), peaks_x, x2, np.array(y_line_score), peaks_y)
    plt.savefig(f'{outdir}{name}_decisions.png')
    plt.close()
    #cv2.destroyAllWindows()
    #super_y = [[line, num]...]
    #print(id_info)
    return num_modules, num_cor, img_vert_lines, super_x, smod_size

In [29]:
def get_img_info(img_start, img_end) :
    #Finds image name in drone data and returns the useful bits in order of images taken
    #theres some play in the data which could be accounted for here
    #TODO:validation with epoch
    csv_file = "/home/dm3315/Documents/SK/PMT_learning/images/Raw/SK_TOW_2020_Image_Database-ImageDroneData.csv"
    rows = []
    #img_start = 4147
    #img_end = 4656
    
    with open(f'{csv_file}') as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
    #print(readCSV)
        for row in readCSV:
            rows.append(row)
    
    rows = rows[img_start - 1:img_end]

    if len(rows)%10 != 0 :
        print(broken)
        exit()
    img_order = []
    img_info = {}
    
    #Have to manually count index of information and add to dict
    for i in rows :
        img_info[f'{i[2][1:4]}'] = {'survey':i[0],'epoch':i[7],'depth':i[15], 'yaw':i[17], 'pitch':i[18], 'roll':i[19] }
        img_order.append(f'{i[2][1:4]}')

    return img_info, np.unique(img_order)

## Reprojection code

In [30]:
def reproject(init_vert_lines, init_hor_lines, smod_width, init_yaw, end_yaw ) :
    #function projects lines from one image to next, 
    #TODO: do same with depth for moving down tank to next ring
    
    global smod_track_lines
    global smod_track_num
    global ring_start_yaw
    #TODO: line up images by looking at horizontal lines and label them
    #todo: adjust guess by rescaling shift based on change in super module size
    #TODO: account for getting back to start of ring

    
    
    just_vert_lines = []
    for vert_line in range(len(init_vert_lines)) :
        just_vert_lines.append(init_vert_lines[vert_line][0])
    

    #hor_smod_px = np.average(np.diff(init_hor_lines))
    
   # scale_rat = vert_smod_px/hor_smod_px
    smod_len = 9.6 #degrees
    delta_y = int(end_yaw) - int(init_yaw)
    
    if abs(delta_y) > 340:
        mag_delta_y = abs(360-delta_y)
        sign_delta_y = delta_y/abs(delta_y)
        delta_y = mag_delta_y * sign_delta_y
        
    shift_px = (delta_y / smod_len) #* scale_rat  #this is the fraction of a super module moved to the next image
    
    

    print(f'smod_size (repo):{smod_width}')
    #print(f'aspect_ratio:{scale_rat}')
    print(f'shift:{shift_px}')
    print(f'initial lines:{init_vert_lines}')
    
    projected_vert_lines = []
    if shift_px < 0 :
        for i in range(len(init_vert_lines)) :
            projected_vert_lines.append([(init_vert_lines[i][0]/smod_width) + shift_px, init_vert_lines[i][1]] )
    else :
        for i in range(len(init_vert_lines)) :
            projected_vert_lines.append([(init_vert_lines[i][0]/smod_width) - shift_px, init_vert_lines[i][1]] )
            

    #return fraction of pmt moved
    return [projected_vert_lines, smod_width]

# RUN HERE

In [32]:
#Need segemnted files for input (could be done with just points maybe?
#need the csv file with information
#Raw images useful for overlay
#outputs txt file with positions of pmts and super modules horizontally and vertically
#BUGS: double counting

import time 
start_time = time.time()


############### SETUP ##################################
method  ='guess_meth'    #manual_intervention
seg_files = 'ring_avg_all'

#For manual Override
global supervised 
supervised = False

global use_undistort
use_undistort = True

work_dir = f"/home/dm3315/Documents/SK/PMT_learning/images/datasets/substacks_far/output/{seg_files}/"
#out_dir = f"/home/dm3315/Documents/SK/PMT_learning/images/ring_reco/four_gaps/{method}/"
out_dir = f"/home/dm3315/Documents/SK/PMT_learning/images/datasets/substacks_far/output/{seg_files}/smodule/{method}/"

global raw_img_dir 
raw_img_dir = '/home/dm3315/Documents/SK/PMT_learning/images/Raw/neutsrv2.triumf.ca/neutshare/superk/TOW-Feb2020/Processed/BarrelSurveyRings_TB3/BarrelSurveyRings_median/ring_avg_all/'

#indir = "/home/dm3315/Documents/SK/PMT_learning/images/Raw/ring_avg/"


#info, order = get_img_info(4457, 4656)
info, order = get_img_info(4147, 4656)
#info, order = get_img_info(3117, 3166) #korean injectors
#info, order = get_img_info(2937, 3466)
#info, order = get_img_info(4557, 4656)
#print(info)

############################################################
f = []
for (dirpath, dirnames, filenames) in os.walk(f'{work_dir}'):
    f.extend(filenames)
    break
f = [x[:-4] for x in f]


tot_mods = 0
tot_cor = 0
img_num = 1

#for histograms
max_width_data = {'true':[], 'false':[]}
average_max_data = {'true':[], 'false':[]}

#global variables that may not be used
smod_track_num = 0
smod_count_num = 0
prev_img_end_num = 0
prev_hor_end_num = 0
smod_count_lines = [] #just counting lines
smod_track_lines = [] #applying unique numbers
failed_imgs = []

#get list of images in order they were taken (i.e. going round the ring)
os.chdir(f"{work_dir}")

#clear files before overwrite
pmt_info_file = open(f'{out_dir}/pmt_loc.txt', 'w')
pmt_info_file.close()
pmt_label_file = open(f'{out_dir}/pmt_loc_label.txt', 'w')
pmt_info_file.close()
num_images = len(order) - 1
print(f'num images:{num_images}')

#loop em
i = 0
while i < num_images :

    #Get list of image names from files 
    if len(glob.glob(f"{order[i]}*")) > 1 :
        print(f'Duplicate images: {glob.glob(f"{order[i]}*")}')
        continue
        
    filename = glob.glob(f"{order[i]}*")[0]
    
    if len(glob.glob(f"{order[i+1]}*")) == 0 :
        order = np.delete(order, i+1)
        num_images -= 1
    next_filename = glob.glob(f"{order[i+1]}*")[0]
    print(f'\n Processing image {img_num} of {len(order)}: {filename[:-4]}')

    
    #TODO:change this to do multiple rings
    if img_num == 1 :
        smod_shift = [[], 1]
        ring_start_yaw = None
    num, num_cor, vert, hor, smod_width = find_super_modules(work_dir, out_dir, filename[:-4], smod_shift)
    tot_mods += num
    tot_cor += num_cor
    print(f'num_cor = {tot_cor}')
    
    #TODO:decide if ring is finished:
    
           #get start of ring and catch end of ring (end doesnt work)
    text_slice = 3 #for changes in image names ~(hopefully temporary
    if seg_files=='ring_avg' or seg_files=='ring_avg_all':
        yaw_1 = int(info[filename[:text_slice ]]['yaw'])
        yaw_2 = int(info[next_filename[:text_slice ]]['yaw'])
        depth_1 = info[filename[:text_slice ]]['depth']
        depth_2 = info[next_filename[:text_slice ]]['depth']
        if yaw_2 - yaw_1 < 0 :
            if ring_start_yaw == None :
                ring_start_yaw = yaw_1
            else :
                if (abs(yaw_1 - ring_start_yaw < 10)) and (yaw_1 < ring_start_yaw) and (img_num > 20) :
                    print(f'ring complete at depth :{depth_1}')
                    #ring_start_yaw = None
                    #TODO: get to next ring depth, project initial img lines back to last img lines and label
                    #TODO: reproject horizontal line labels to next depth
                    #TODO: other way around ring?
                    
        
        #Reproject image horizontally to next one
        print(f'yaws: {yaw_1}, {yaw_2}')
        print(f'names: {order[i]}, {order[i+1]}')
        #scaled by horizontal spacing, basically aspect ratio
        smod_shift = reproject(vert, hor, smod_width, yaw_1, yaw_2 )
        print(f'smods moved:{smod_shift}')
    i += 1
    img_num += 1 
    
pmt_info_file.close()
print(f"found {prev_img_end_num} Super modules horizontally in ring. {tot_cor} of 64 are correct")
print(f'Images with failed labels: {failed_imgs}')

print(f'time taken = {time.time() - start_time} seconds')

num images:51

 Processing image 1 of 52: 755_pred_substacks_far_unet.19
Guess: 7 horizontal gaps above 1948.2153191489363, 11 vertical above 1206.4405263157894
scores for compare None [10000. 10000.] range(2, 10, 4)
scores for compare None [10000. 10000.] range(3, 10, 4)
scores for compare None [10000. 10000. 10000.] range(0, 10, 4)
scores for compare None [10000. 10000. 10000.] range(1, 10, 4)
hyp Scores:[0. 0. 0. 0.]
smod sizes smod/pmt: 1411.492890995261, 1432.7540810953135
[1048.3922487223167, 1073]
[1607.846234860453, 1603]
[3033.0963665086892, 3010]
Found 1 horizontal gaps
Found 1 vertical gaps
num_cor = 3
yaws: 165, 159
names: 755, 756
smod_size (repo):1411.492890995261
shift:-0.625
initial lines:[[3.10110585e+02 1.00000000e+00]
 [1.70784623e+03 2.00000000e+00]
 [3.13309637e+03 3.00000000e+00]]
smods moved:[[[-0.4052960351436806, 1.0], [0.5849573761623621, 2.0], [1.5947039648563193, 3.0]], 1411.492890995261]

 Processing image 2 of 52: 756_pred_substacks_far_unet.19
Guess: 7 ho

KeyboardInterrupt: 

# Testing below here

In [ ]:
#histograms

In [ ]:
num_rem = 2
a = [1, 2, 3, 4, 5, 6, 7]

b = a[num_rem:-num_rem]
print(b)

In [ ]:
#dtype = [('hyp', int), ('max_w', float), ('avg_lo', float)]
#print(len(dtype))
#print(type(dtype[0][0]))
#print(np.zeros(4))n

#line_labels = np.array([1, 2, 0])
#a = np.nonzero(line_labels)
#print(a[0])

gaps = [20, 30., 40, 50,60, 70, 80, 90]
num = 2
a = np.full(len(range((3+num)%4,len(gaps),4)), 10000., dtype=float)
for i in range((3+num)%4,len(gaps),4) :
    print(i)
print(a)

In [ ]:
best_override = input('Set new lines (between 0 and 3):')
print(best_override.lower())
print(type(best_override.lower()))

In [ ]:
inpath = '/home/dm3315/Documents/SK/PMT_learning/images/Raw/large_labeled/frames/B5310.png'   
imge = cv2.imread(f'{inpath}')
out = UndistortImage(imge)

cv2.imwrite("/home/dm3315/Documents/SK/PMT_learning/images/Raw/undistorted/orig.png", imge)
cv2.imwrite("/home/dm3315/Documents/SK/PMT_learning/images/Raw/undistorted/undist.png", out)
#cv2.waitKey(0) 

In [ ]:
import os
#indir = "/home/dm3315/Documents/SK/PMT_learning/images/Raw/ring_avg/"
f = []
for (dirpath, dirnames, filenames) in os.walk(f'{indir}'):
    f.extend(filenames)
    break

for k in f :
    orig = cv2.imread(f"{indir}{k}")
    undist = UndistortImage(orig)
    cv2.imwrite(f"{indir}udist/{k}", undist)

In [ ]:
def ConvertMasksBGR(BGR_img) :
    lower_pmts = np.array([254, 0, 0], dtype = "uint16")
    upper_pmts = np.array([255,0,0], dtype = "uint16")
    pmt_mask = cv2.inRange(BGR_img, lower_pmts, upper_pmts)
    pmt_mask[pmt_mask == 255] = 1
    return pmt_mask

inpath = "/home/dm3315/Documents/SK/PMT_learning/images/datasets/bolt_pmt_blur/output/ring_avg/B7550546.png_pred_noverlay.png"
img = cv2.imread(f"{inpath}")
print(np.unique(img))

pmts = ConvertMasksBGR(img)
#pmts_gray = cv2.cvtColor(pmts, cv2.COLOR_BGR2GRAY)
print(np.unique(pmts))
cv2.imshow("img", pmts)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
info = get_img_info(4147, 4656)
print(info)
yaw = [] 
for i in info :
    print(i['name'])
    yaw.append(int(i['yaw']))
print(np.sort(np.unique(yaw)))
print(len(np.unique(yaw)))



In [ ]:
#apply convolution 

#prepare the 5x5 shaped filter
kernel = np.array([[1, 1, 1, 1, 1], 
                   [1, 1, 1, 1, 1], 
                   [1, 1, 1, 1, 1], 
                   [1, 1, 1, 1, 1], 
                   [1, 1, 1, 1, 1]])
kernel = np.round(kernel/sum(kernel))

#filter the source image
img_rst = cv2.filter2D(img_in,-1,kernel)

#save result image
cv2.imwrite('/home/dm3315/Documents/SK/PMT_learning/images/conv.png',img_rst)


In [ ]:
#loop through and view raw images
#BUGS: double counting
import os
import cv2
import csv
import numpy as np
import glob

idir = '/home/dm3315/Documents/SK/PMT_learning/images/Raw/neutsrv2.triumf.ca/neutshare/superk/TOW-Feb2020/Processed/BarrelSurveyRings_TB3/BarrelSurveyRings_median/ring_avg_all/'

f = []
for (dirpath, dirnames, filenames) in os.walk(f'{idir}'):
    f.extend(filenames)
    break
#f = [x[:-4] for x in f]

info, order = get_img_info(4147, 4656)

print(order)

for i in range(len(order)) :
    filename = f'{idir}{order[i]}.jpg'
    print(filename)
    cv2.namedWindow(order[i],cv2.WINDOW_NORMAL) ####################### Added cv2.WINDOW_NORMAL flag to allow to resize window.
    ##cv2.moveWindow(filename, 40, 30) ##40 and 30 are x and y coordinates on the screen
    cv2.moveWindow(order[i], 500, 0)
    cv2.resizeWindow(order[i], 1200, 900)
    print(info[f'{order[i]}']['yaw'])
    img = cv2.imread(filename) ##read the image specified in the input
    if i == 0 :
        cv2.imshow(order[i],img)   
    else :
        cv2.namedWindow(order[i-1],cv2.WINDOW_NORMAL) ####################### Added cv2.WINDOW_NORMAL flag to allow to resize window.
        ##cv2.moveWindow(filename, 40, 30) ##40 and 30 are x and y coordinates on the screen
        cv2.moveWindow(order[i-1], 50, 0)
        cv2.resizeWindow(order[i-1], 1200, 900)
        next_img = cv2.imread(f'{idir}{order[i-1]}.jpg')
        cv2.imshow(order[i],img)
        cv2.imshow(order[i-1],next_img)     
    cv2.waitKey(0)
    cv2.destroyAllWindows()



In [ ]:

val_coords = {'755':{'vert_lines':[[1703, 1], [3110, 2]], 'hor_lines':[1273]},
'756': {'vert_lines':[[1933, 2]], 'hor_lines':[1299]},
'757': {'vert_lines':[[910, 2], [2293, 3]], 'hor_lines':[1266]},
'758': {'vert_lines':[[1450, 3], [2750, 4]], 'hor_lines':[1300]},
'759': {'vert_lines':[[1820, 4], [3175, 5]], 'hor_lines':[1270]},
'760': {'vert_lines':[[2000, 5], [3245, 6]], 'hor_lines':[1883]},
'761': {'vert_lines':[ [2345, 5]], 'hor_lines':[1300]},
'762': {'vert_lines':[[1270, 5], [2576, 6]], 'hor_lines':[1266]},
'763': {'vert_lines':[[1706, 6], [3026, 7]], 'hor_lines':[2203]},
'764': {'vert_lines':[[690, 6], [2100, 7]], 'hor_lines':[1259]},       
'765': {'vert_lines':[[1050, 7], [2450, 8]], 'hor_lines':[1259]},
'766': {'vert_lines':[[1463, 8], [2886, 9]], 'hor_lines':[1256]},
'767': {'vert_lines':[[1800, 9], [3129,10]], 'hor_lines':[1250]},
'768': {'vert_lines':[[729, 9], [2100, 10], [3460, 11]], 'hor_lines':[1263]},     
'769': {'vert_lines':[[1070, 10], [2383, 11]], 'hor_lines':[1266]},
'770': {'vert_lines':[[1383, 11], [2733, 12]], 'hor_lines':[1276]},
'771': {'vert_lines':[[1575, 12], [2883, 13]], 'hor_lines':[1250]},
'772': {'vert_lines':[[2046, 12], [3350, 13]], 'hor_lines':[1263]},
'773': {'vert_lines':[[950, 12], [2306, 13]], 'hor_lines':[1246]},
'774': {'vert_lines':[[1463, 13], [2783, 14]], 'hor_lines':[1256]},
'775': {'vert_lines':[[1789, 14], [3033, 15]], 'hor_lines':[1236]},
'776': {'vert_lines':[[700, 15], [1956, 16], [3416, 17]], 'hor_lines':[1273]},
'777': {'vert_lines':[[1000, 16], [2400, 17]], 'hor_lines':[1253]},
'778': {'vert_lines':[[1370, 17], [2904, 18]], 'hor_lines':[1845]},
'779': {'vert_lines':[[1770, 18], [3316, 19]], 'hor_lines':[1220]},
'780': {'vert_lines':[[641, 19], [2123, 20]], 'hor_lines':[1250]},
'781': {'vert_lines':[[950, 20], [2441, 21]], 'hor_lines':[1250]},
'782': {'vert_lines':[[1370, 21], [2936, 22]], 'hor_lines':[1236]}, #these past few have been really good
'783': {'vert_lines':[[1787, 22], [3320, 23]], 'hor_lines':[2000]},
'784': {'vert_lines':[[1354, 23], [2150, 24]], 'hor_lines':[1243]}, #differs by 20 deg so mayb 24, 25
'785': {'vert_lines':[[945, 24], [2483,25]], 'hor_lines':[1237]},              
'786': {'vert_lines':[[1293, 25], [2809, 26]], 'hor_lines':[1250]},
'787': {'vert_lines':[[1630,26], [3120, 27]], 'hor_lines':[1250]}, #maybe wronf
'788': {'vert_lines':[[730, 26], [2173, 27]], 'hor_lines':[1219]},
'789': {'vert_lines':[[750, 28], [2360, 29]], 'hor_lines':[1237]}, #18 degrees
'790': {'vert_lines':[[1203, 29], [2810, 30]], 'hor_lines':[1256]},
'791': {'vert_lines':[[1620, 30], [3283, 31]], 'hor_lines':[1250]},
'791': {'vert_lines':[[579, 30], [2220, 31]], 'hor_lines':[1987]},  #Double check!!!!!!!!!!!!!!!!!!!!!
'792': {'vert_lines':[[560, 30], [2150, 31], [3950, 32]], 'hor_lines':[1266]}, #half super module
'793': {'vert_lines':[[816, 31], [1600, 32], [3154, 33]], 'hor_lines':[1266]}, #half super module
'794': {'vert_lines':[[600, 32], [2083, 33]], 'hor_lines':[1250]},
'795': {'vert_lines':[[908, 33], [2412, 34]], 'hor_lines':[1300]},
'796': {'vert_lines':[[1250, 34], [2816, 35]], 'hor_lines':[1250]},
'797': {'vert_lines':[[1725, 35], [3191, 36]], 'hor_lines':[1240]},
'798': {'vert_lines':[[570, 35], [2050, 36]], 'hor_lines':[1250]},
'799': {'vert_lines':[[1100, 35], [2450, 36]], 'hor_lines':[1250]},
'800': {'vert_lines':[[1429, 36], [2700, 37]], 'hor_lines':[1236]},
'901': {'vert_lines':[[1720, 37], [3158, 38]], 'hor_lines':[1236]},
'902': {'vert_lines':[[570,38], [1578, 39]], 'hor_lines':[1279]},    #64 lines above here
'903': {'vert_lines':[[937,39], [2370, 40]], 'hor_lines':[1279]},    #64 lines above here
'904': {'vert_lines':[[1250,40], [2620, 41]], 'hor_lines':[1279]},    #64 lines above here
'905': {'vert_lines':[[1687,40], [3000, 41 ]], 'hor_lines':[1279]},    #64 lines above here

    }

In [ ]:
#old super module decision which kind of worked
            '''
            if (super_y[line]+vert_margin > max_prev_line + margin) and line !=0  :
                if line_labels[1, line-1] > 0
                    line_labels[1, line] = line_labels[1, line-1] +1
                   # img_vert_lines.append([super_y[line], prev_img_end_num]) #or img_vert_line[-1] +1 ?
                   # num_assigned = True
                    lines_numbered += 1

            #if a line is not assigned a number give it the past number
            if not num_assigned :
                failed_imgs.append(f'{base_name}')
                
                # Take the number of nearest predicted line to the left
                for prev_line in range(len(cor_lines)):
                    if cor_lines[prev_line] < super_y[line] + vert_margin :
                        failed_line_num = cor_num[prev_line]
                        num_assigned = True
                if num_assigned :
                 #   img_vert_lines.append([super_y[line], failed_line_num ])
                #if there isnt one (there should always be really) but in case drone has gone a long way in opposite direction
                #label it relative to other lines in the image
                
            if not num_assigned :
                print('Shit.')
               # img_vert_lines.append([super_y[line], 0 ])
            '''    
#

In [ ]:
#Gets the angle of pmts in the image to better do the pixel scan and get scores
#not really necessary so abandoned, may be useful later, implementation here is not good!!
#UPDAATE this code to not use circles, maximise score, may be longer but faaar better
def get_angle(in_img) :
    
    img = in_img.copy()
    
    gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img_size = gray_img.shape
    gray_img[gray_img > 0] = 100
    gray_img = gray_img[500:-500, :] 
   # print(np.unique(gray_img))
    edges = cv2.Canny(gray_img,0,100)
    circles = cv2.HoughCircles(edges,cv2.HOUGH_GRADIENT,dp=1,
                           minDist=200,
                           param1=100,
                           param2=20,
                           minRadius=140,
                           maxRadius=200
                          )
        #print(circles)
        
    #x, y data for circle centers
    circles = np.uint16(np.around(circles))
    circ_x = []
    circ_y = []
    for i in circles[0, :] :
        cv2.circle(img,(i[0],i[1]),i[2],(0,255,0),2)
        #print(i[0], i[1])
        circ_x.append(i[0])
        circ_y.append(i[1])
  #  print(np.min(circ_y))
  #  print(np.max(circ_y))
    
    
    #Get the first row of circles
    circ_x_first = []
    circ_y_first = []
    min_y = np.min(circ_y)
    
    #for drawing circles
    for i in range(len(circ_y)) :
        if circ_y[i] <  2*min_y :
         #   circ_y.pop(i)
          #  circ_x.pop(i)
            circ_x_first.append(circ_x[i])
            circ_y_first.append(circ_y[i])
                                
    for i in range(len(circ_x_first)) :
        cv2.circle(img, (circ_x_first[i], circ_y_first[i]), 100, (0,255,0), -1)
    
    #straight line function
    def func(x, a, b) :
        return a*x + b
    popt, pcov = curve_fit(func, circ_x_first, circ_y_first)
    
    
   #print(popt)
    #print(circ_x)
    #print(circ_y)
    #cv2.imshow('img', img)
    #cv2.waitKey(0)
    return popt[0]